# Binary Matrix Factorization

Requires the following packages to be installed in the environment: 

```
- numpy
- scikit-learn
- matplotlib
- opencv-python
```



In [ ]:
import numpy as np
from time import perf_counter
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.neighbors import NearestNeighbors

def nearest_neighbor(points, data, n_neighbors = 1, p=2):
    """Function for finding the nearest neighbor of points in a dataset.
    
    Used to replace the k-means Steiner points with the nearest neighbor
    in the dataset.
    
    Wraps the scipy nearest neighbor class."""
    
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto', p=p, n_jobs=-1).fit(data)
    _, indices = nbrs.kneighbors(points)
    #print(np.bincount(indices.flatten()))
    
    return data[indices.flatten()]

In [ ]:
from scipy.spatial.distance import minkowski

def l_p_norm(v, p=2):
    return np.power(np.linalg.norm(v, ord=p), p)    

def l_p_dist(u, v, p=2):
    return np.power(np.linalg.norm(u-v, ord=p), p)

# vector valued functions
def l_1_dist(u, v):
    return l_p_dist(u,v,1)

def l_2_dist(u, v):
    return l_p_dist(u,v,2)

def l_5_dist(u, v):
    return l_p_dist(u,v,5)

# Kumar et al.'s algorithm for Binary Matrix Factorization

In [421]:
from sklearn.cluster import KMeans, k_means

class BMFKMeans:
    '''The proposed algorithm of Kumar et al. (2019) to obtain a low-rank binary matrix
    that approximates a given matrix.'''
    
    def __init__(self, k, max_iter=300):
        self.k = k
        self.max_iter = max_iter

    def __fit(self, data):
        #t0 = time()
        kmeans = KMeans(init="k-means++", n_clusters=self.k, max_iter=self.max_iter, n_init='auto')
        kmeans.fit(data) # get clusters
        #print("Elapsed time for k-means: %f" % (time() - t0))
        #cluster_centers_, labels_, inertia_ = k_means(data, self.k, n_init='auto', max_iter=self.max_iter)
        
        #t1 = time()
        self.centers_ = nearest_neighbor(kmeans.cluster_centers_, data, p=1)
        self.low_rank_ = nearest_neighbor(data, self.centers_) # self.centers_[kmeans.labels_]
        #print("Elapsed time for nearest-neighbors: %f" % (time() - t1))
        
    def fit(self, data, labels = []):
        if (data.ndim == 2):
            self.__fit(data)
            return self
        else: 
            raise ValueError("Can not factorize a dataset of shape % s." % str(data.shape))
            
    def results(self):
        return self.low_rank_
    
    def score(self, data, label):
        return np.square(np.linalg.norm(data - self.low_rank_, ord='fro'))

# Our algorithm for Binary Matrix Factorization

In [ ]:
from sklearn.cluster import KMeans, k_means

class BMFKMeans_plus:
    '''The proposed algorithm of Kumar et al. (2019) to obtain a low-rank binary matrix
    that approximates a given matrix - with post processing.'''
    
    def __init__(self, k, max_iter=300):
        self.k = k
        self.max_iter = max_iter

    def __fit(self, data):
        #t0 = time()
        kmeans = KMeans(init="k-means++", n_clusters=self.k, max_iter=self.max_iter, n_init='auto')
        kmeans.fit(data) # get clusters
        #print("Elapsed time for k-means: %f" % (time() - t0))
        #cluster_centers_, labels_, inertia_ = k_means(data, self.k, n_init='auto', max_iter=self.max_iter)
        
        #t1 = time()
        self.centers_ = nearest_neighbor(kmeans.cluster_centers_, data, p=1)
        self.low_rank_ = self.optimize(data)
        #print("Elapsed time for nearest-neighbors: %f" % (time() - t1))
        
    def fit(self, data, labels = []):
        if (data.ndim == 2):
            self.__fit(data)
            return self
        else: 
            raise ValueError("Can not factorize a dataset of shape % s." % str(data.shape))
            
    def results(self):
        return self.low_rank_
    
    def score(self, data, label):
        return np.linalg.norm(data - self.low_rank_, ord='fro')
    
    def optimize(self, data):
        combinations = []
        # enumerate all possible linear combinations
        for i in range(2**self.k):
            #if i % 10000 == 0 and i > 0:
            #    print(i)
            bits = []
            for j, c in enumerate(bin(i)[:1:-1], 1):
                if c == '1':
                    bits.append(j-1)
            combinations.append(np.sum(self.centers_[bits], axis=0))
        combinations = np.mod(np.array(combinations), 2)
        combinations = np.unique(combinations, axis=0)
        # print("Obtained %d unique combinations." % combinations.shape[0])
        return nearest_neighbor(data, np.array(combinations), p=1)
        
            
            

In [ ]:
from sklearn.cluster import KMeans, k_means

class BMFKMeans_plus_int_1:
    '''The proposed algorithm of Kumar et al. (2019) to obtain a low-rank binary matrix
    that approximates a given matrix, with post-processing.'''
    
    def __init__(self, k, max_iter=300):
        self.k = k
        self.max_iter = max_iter

    def __fit(self, data):
        #t0 = time()
        kmeans = KMeans(init="k-means++", n_clusters=self.k, max_iter=self.max_iter, n_init='auto')
        kmeans.fit(data) # get clusters
        #print("Elapsed time for k-means: %f" % (time() - t0))
        #cluster_centers_, labels_, inertia_ = k_means(data, self.k, n_init='auto', max_iter=self.max_iter)
        
        #t1 = time()
        self.centers_ = nearest_neighbor(kmeans.cluster_centers_, data, p=1)
        self.low_rank_ = self.optimize(data)
        #print("Elapsed time for nearest-neighbors: %f" % (time() - t1))
        
    def fit(self, data, labels = []):
        if (data.ndim == 2):
            self.__fit(data)
            return self
        else: 
            raise ValueError("Can not factorize a dataset of shape % s." % str(data.shape))
            
    def results(self):
        return self.low_rank_
    
    def score(self, data, label):
        return l_p_norm((data - self.low_rank_).flatten(), p=1)
    
    def optimize(self, data):
        combinations = []
        # enumerate all possible linear combinations
        for i in range(2**self.k):
            #if i % 10000 == 0 and i > 0:
            #    print(i)
            bits = []
            for j, c in enumerate(bin(i)[:1:-1], 1):
                if c == '1':
                    bits.append(j-1)
            combinations.append(np.sum(self.centers_[bits], axis=0))
        # combinations = np.mod(np.array(combinations), 2)
        combinations = np.unique(combinations, axis=0)
        # print("Obtained %d unique combinations." % combinations.shape[0])
        return nearest_neighbor(data, np.array(combinations), p=1)
        
            
            

In [ ]:
from sklearn.cluster import KMeans, k_means

class BMFKMeans_plus_int_2:
    '''The proposed algorithm of Kumar et al. (2019) to obtain a low-rank binary matrix
    that approximates a given matrix, with post-processing.'''
    
    def __init__(self, k, max_iter=300):
        self.k = k
        self.max_iter = max_iter

    def __fit(self, data):
        #t0 = time()
        kmeans = KMeans(init="k-means++", n_clusters=self.k, max_iter=self.max_iter, n_init='auto')
        kmeans.fit(data) # get clusters
        #print("Elapsed time for k-means: %f" % (time() - t0))
        #cluster_centers_, labels_, inertia_ = k_means(data, self.k, n_init='auto', max_iter=self.max_iter)
        
        #t1 = time()
        self.centers_ = nearest_neighbor(kmeans.cluster_centers_, data, p=1)
        self.low_rank_ = self.optimize(data)
        #print("Elapsed time for nearest-neighbors: %f" % (time() - t1))
        
    def fit(self, data, labels = []):
        if (data.ndim == 2):
            self.__fit(data)
            return self
        else: 
            raise ValueError("Can not factorize a dataset of shape % s." % str(data.shape))
            
    def results(self):
        return self.low_rank_
    
    def score(self, data, label):
        return l_p_norm((data - self.low_rank_).flatten(), p=2)
    
    def optimize(self, data):
        combinations = []
        # enumerate all possible linear combinations
        for i in range(2**self.k):
            #if i % 10000 == 0 and i > 0:
            #    print(i)
            bits = []
            for j, c in enumerate(bin(i)[:1:-1], 1):
                if c == '1':
                    bits.append(j-1)
            combinations.append(np.sum(self.centers_[bits], axis=0))
        # combinations = np.mod(np.array(combinations), 2)
        combinations = np.unique(combinations, axis=0)
        # print("Obtained %d unique combinations." % combinations.shape[0])
        return nearest_neighbor(data, np.array(combinations), p=2)
        
            
            

In [ ]:
from sklearn.cluster import KMeans, k_means

class BMFKMeans_plus_int_5:
    '''The proposed algorithm of Kumar et al. (2019) to obtain a low-rank binary matrix
    that approximates a given matrix, with post-processing.'''
    
    def __init__(self, k, max_iter=300):
        self.k = k
        self.max_iter = max_iter

    def __fit(self, data):
        #t0 = time()
        kmeans = KMeans(init="k-means++", n_clusters=self.k, max_iter=self.max_iter, n_init='auto')
        kmeans.fit(data) # get clusters
        #print("Elapsed time for k-means: %f" % (time() - t0))
        #cluster_centers_, labels_, inertia_ = k_means(data, self.k, n_init='auto', max_iter=self.max_iter)
        
        #t1 = time()
        self.centers_ = nearest_neighbor(kmeans.cluster_centers_, data, p=1)
        self.low_rank_ = self.optimize(data)
        #print("Elapsed time for nearest-neighbors: %f" % (time() - t1))
        
    def fit(self, data, labels = []):
        if (data.ndim == 2):
            self.__fit(data)
            return self
        else: 
            raise ValueError("Can not factorize a dataset of shape % s." % str(data.shape))
            
    def results(self):
        return self.low_rank_
    
    def score(self, data, label):
        return l_p_norm((data - self.low_rank_).flatten(), p=5)
    
    def optimize(self, data):
        combinations = []
        # enumerate all possible linear combinations
        for i in range(2**self.k):
            #if i % 10000 == 0 and i > 0:
            #    print(i)
            bits = []
            for j, c in enumerate(bin(i)[:1:-1], 1):
                if c == '1':
                    bits.append(j-1)
            combinations.append(np.sum(self.centers_[bits], axis=0))
        # combinations = np.mod(np.array(combinations), 2)
        combinations = np.unique(combinations, axis=0)
        # print("Obtained %d unique combinations." % combinations.shape[0])
        return nearest_neighbor(data, np.array(combinations), p=5)
        
            
            

In [424]:
bmf = BMFKMeans(15)
bmf.fit(congress)
np.sqrt(bmf.score(congress, congress))


30.099833886584822

# Coreset Constructions

In [ ]:
class LWCoreset:
    '''An algorithm for computing light-weight coresets, based on the paper
    Scalable k-means clustering via lightweight coresets by Bachem, Lucic and Krause
    
    This gives an additive-multiplicative guarantee on the costs of k-means
    clustering.'''
    def __init__(self, m):
        self.m = m
    
    def fit(self, data):
        if data.ndim != 2:
            raise ValueError("Dataset needs to be of dimension 2, is of dimension % s" % data.ndims)
        
        n, d = data.shape
        
        #t0 = perf_counter()
        mean = np.mean(data, axis=0)
        dists = np.square(np.linalg.norm(data - mean, axis=1))
        
        self.p = (1.0 / n + dists / np.sum(dists))/2.0
        #print("Calculated Distribution in %f s" % (perf_counter() - t0))

        #t0 = perf_counter()
        self.indices = np.random.choice(n, size=self.m, replace=True, p=self.p)
        #print("Sampled Indices in %f s" % (perf_counter() - t0))

        #t0 = perf_counter()
        self.coreset = data[self.indices]
        self.weights = 1.0 / (self.m * self.p[self.indices])
        #print("Selected Coreset and Weights in %f s" % (perf_counter() - t0))
        
        return self
        
    def result(self):
        return (self.coreset, self.weights)       

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import kmeans_plusplus
from sklearn.cluster import KMeans
from sklearn.utils.extmath import row_norms

class KMeansCoreset:
    def __init__(self, delta, k):
        self.delta = delta
        self.k = k
        #self.d = d
        
    def fit(self, X, size):
        n, d = X.shape
        
        # calculate squared norms of points
        #t0 = perf_counter()
        model = KMeans(self.k)
        norms = row_norms(X, squared=True)
        cs = model._init_centroids(X, init='k-means++',
                                  x_squared_norms=norms, random_state=np.random.RandomState(seed=0))
        #print("D2 sampling took %f" % (perf_counter() - t0))
        
        
        dist = row_norms(X - nearest_neighbor(X, cs), squared=True)
        cs_cost = np.sum(dist)
        c_total = cs_cost / n      
        
        nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(cs)
        _, indices = nbrs.kneighbors(X) # euclidean distance and labels
        indices = indices.flatten()
        
        keys, counts = np.unique(indices, return_counts=True)
        cluster_sizes = np.array(counts)
        
        
        if c_total < 0.00001:
            #print("Coreset centers cover whole space.")
            self.coreset = cs
            self.weights = np.ones(cs.shape)/ self.k
            return self
        
        # calculate sensitivity
        cluster_cost = np.bincount(indices, dist)
        cluster_sizes_ = cluster_sizes[indices]
        
        alpha = 16 * np.log2(self.k) + 32
        s = alpha / c_total * dist + (2 * alpha * cluster_cost[indices] / c_total + 4 * n) / cluster_sizes_
        
        
        p = s / np.sum(s)
        
        self.indices = np.random.choice(n, size = size, p = p)
        self.coreset = X[self.indices]
        self.weights = 1.0 / (size * p[self.indices])
        
        return self
        
    def result(self):
        return (self.coreset, self.weights)

In [ ]:
from sklearn.cluster import KMeans, k_means

class BMFCoreset:
    '''Using a Coreset for BMF'''
    
    def __init__(self, k, size, max_iter=300):
        self.k = k
        self.size = size
        self.max_iter = max_iter

    def __fit(self, data):
        n, d = data.shape
        
        #kmcs = LWCoreset(self.size).fit(data)
        #t0 = perf_counter()
        kmcs = KMeansCoreset(0.99, self.k).fit(data, self.size)
        cs, ws = kmcs.result()
        #print("Calculated Coreset in %f s" % (perf_counter() - t0))
        if cs.shape[0] > k:
            kmeans = KMeans(init="k-means++", n_clusters=self.k, max_iter=self.max_iter, n_init='auto')
            kmeans.fit(cs, sample_weight=ws) # get clusters
            self.centers_ = nearest_neighbor(kmeans.cluster_centers_, data)
        else:
            self.centers_ = cs
        #print("Elapsed time for k-means: %f" % (time() - t0))
        #cluster_centers_, labels_, inertia_ = k_means(data, self.k, n_init='auto', max_iter=self.max_iter)
        
        #t1 = time()
        #self.low_rank_ = nearest_neighbor(data, self.centers_)
        self.low_rank_ = self.optimize(data)

        #self.low_rank_ = centers_[labels_]
        #print("Elapsed time for nearest-neighbors: %f" % (time() - t1))
        
    def fit(self, data, labels = []):
        if (data.ndim == 2):
            self.__fit(data)
            return self
        else: 
            raise ValueError("Can not factorize a dataset of shape % s." % str(data.shape))
            
    def results(self):
        return self.low_rank_
    
    def score(self, data, label):
        return np.linalg.norm(data - self.low_rank_, ord='fro')
    
    def optimize(self, data):
        combinations = []
        # enumerate all possible linear combinations
        for i in range(2**self.k):
            if i % 10000 == 0 and i > 0:
                print(i)
            bits = []
            for j, c in enumerate(bin(i)[:1:-1], 1):
                if c == '1':
                    bits.append(j-1)
            combinations.append(np.sum(self.centers_[bits], axis=0))
        combinations = np.mod(np.array(combinations), 2)
        combinations = np.unique(combinations, axis=0)
        # print("Obtained %d unique combinations." % combinations.shape[0])
        return nearest_neighbor(data, np.array(combinations))

In [ ]:
from sklearn.cluster import KMeans, k_means

class BMFLWCoreset:
    '''Using a Lightweight Coreset for BMF'''
    
    def __init__(self, k, size, max_iter=300):
        self.k = k
        self.size = size
        self.max_iter = max_iter

    def __fit(self, data):
        n, d = data.shape
        
        #t0 = perf_counter()
        kmcs = LWCoreset(self.size).fit(data)
        cs, ws = kmcs.result()
        #print("Calculated Coreset in %f s" % (perf_counter() - t0))
        
        kmeans = KMeans(init="k-means++", n_clusters=self.k, max_iter=self.max_iter, n_init='auto')
        kmeans.fit(cs, sample_weight=ws) # get clusters
        #print("Elapsed time for k-means: %f" % (time() - t0))
        #cluster_centers_, labels_, inertia_ = k_means(data, self.k, n_init='auto', max_iter=self.max_iter)
        
        #t1 = time()
        self.centers_ = nearest_neighbor(kmeans.cluster_centers_, data)
        #self.low_rank_ = nearest_neighbor(data, self.centers_)
        self.low_rank_ = self.optimize(data)
        
    def fit(self, data, labels = []):
        if (data.ndim == 2):
            self.__fit(data)
            return self
        else: 
            raise ValueError("Can not factorize a dataset of shape % s." % str(data.shape))
            
    def results(self):
        return self.low_rank_
    
    def score(self, data, label):
        return np.linalg.norm(data - self.low_rank_, ord='fro')

    def optimize(self, data):
        combinations = []
        # enumerate all possible linear combinations
        for i in range(2**self.k):
            if i % 10000 == 0 and i > 0:
                print(i)
            bits = []
            for j, c in enumerate(bin(i)[:1:-1], 1):
                if c == '1':
                    bits.append(j-1)
            combinations.append(np.sum(self.centers_[bits], axis=0))
        combinations = np.mod(np.array(combinations), 2)
        combinations = np.unique(combinations, axis=0)
        # print("Obtained %d unique combinations." % combinations.shape[0])
        return nearest_neighbor(data, np.array(combinations))

In [ ]:
from time import perf_counter
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, Binarizer

import warnings
from sklearn.exceptions import ConvergenceWarning

def bench_bmf(bmf, name, k, data):
    """Benchmark to evaluate binary matrix factorization algorithms.
    
    Parameters
    -----------------
    bmf: BMF instance
        a BMF instance with k already set
    name: str
        Name of the algorithm.
    data: ndarray of shape (n_matrices, n_d1, n_d2)
        A dataset of matrices to find low rank binary approximations to.
    """
    results = [name, k]
    warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
    
    # normalize the data
    shape_ = data.shape
    if data.ndim == 3:
        data = data.reshape(shape_[0] * shape_[1], shape_[2])  
    data = Binarizer(threshold=0.33).transform(MinMaxScaler().fit_transform(data)).reshape(shape_)

    
    
    if data.ndim == 2:
        # data is a single matrix
        t0 = perf_counter()
        bmf.fit(data)
        fit_time = perf_counter() - t0
        results.append(fit_time * 1000)
        results.append(bmf.score(data,data))
        results.append(bmf.score(data,data))

    if data.ndim == 3:
        # dataset is a number of matrices
        times = []
        error = []
        i = 0

        for d in data:
            #sleep(1)
            t0 = perf_counter()
            bmf.fit(d)
            fit_time = perf_counter() - t0
            err = bmf.score(d,d)
            #t1 = perf_counter()
            #bmf.fit(d.T)
            #fit_time = fit_time + (perf_counter() - t1)
            #err = min(err_1, bmf.score(d.T, d.T))
            times.append(fit_time * 1000)
            error.append(err)
            i+=1
            if i % 25 == 0:
                print("\r" + name + "-" + str(k) + ": Processed %d elements" % i, end="")
            # Show the results
        
        #m = 10
        #largest = sorted(range(len(error)), key = lambda sub: error[sub])[-m:]
        #print(("Worst %d instances are " % m) + str(largest))
        
        results.append(np.average(times))
        results.append(np.average(error))
        results.append(np.median(error))
    
    formatter_result = (
        "{:8s}\t{:d}\t{:2.3f}ms\t{:.3f}\t{:.3f}          "
    )
    print("\r" + formatter_result.format(*results))
    
    return results

# Synthetic Data Generation

The following functions are used for sampling full rank synthetic data, low rank synthetic data and noisy low rank synthetic data respectively.

In [ ]:
def generate_synthetic_random(n, d, q):
    """Generate a random n by d matrix where each entry is 1 with probability q"""
    return np.random.choice(2, size=(n,d), p=[1-q, q])
    
def generate_synthetic_rank_k(n, d, q, k):
    """Generate a random n by d matrix via two uniformly sampled low rank matrices"""
    A_ = np.random.choice(2, size=(n,k), p=[1-q, q])
    B_ = np.random.choice(2, size=(k,d), p=[1-q, q])
    return np.mod(A_ @ B_, 2)

def generate_noisy_rank_k(n,d,q,k,p):
    """Generate a random n by d matrix via two uniformly sampled low rank matrices, where with probability p we flip a bit"""
    A_ = np.random.choice(2, size=(n,k), p=[1-q, q])
    B_ = np.random.choice(2, size=(k,d), p=[1-q, q])
    C = np.mod(A_ @ B_, 2)
    
    flip = np.random.rand(n,d) < p
    C[flip] = 1 - C[flip]
    return C

# Loading the Datasets

We load the MNIST, ORL, ThyroidRL and Congress Vote Datasets and process them into binary matrices

In [ ]:
# read the mnist dataset

import gzip

f = gzip.open('./datasets/mnist/train-images-idx3-ubyte.gz')
image_size = 28
num_images = 60000

f.read(16)
buf = f.read(image_size**2 * num_images)
mnist = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
mnist = mnist.reshape(num_images, image_size, image_size)

In [ ]:
import cv2
import glob

# we assume that all pgm files live in the same directory 
orl = np.array([cv2.imread(file,0) for file in glob.glob("./datasets/orl/*.pgm*")])
orl_shape = orl.shape
orl = orl.reshape((orl_shape[0] * orl_shape[1] * orl_shape[2], 1))
orl = Binarizer(threshold=0.33).fit_transform(MinMaxScaler().fit_transform(np.array(orl)))
orl = orl.reshape(orl_shape)

In [ ]:
# read congressional votes dataset

def convert_voting(s):
    if s == b'y':
        return 1.0
    if s == b'n':
        return 0.0
    if s == b'?':
        return 0.0
    if s == b'republican':
        return 1.0
    if s == b'democrat':
        return 0.0

from numpy import loadtxt
file = open('./datasets/house-votes-84.csv')
congress = loadtxt(file, delimiter=',', skiprows=1, converters=convert_voting)

In [ ]:
import pandas as pd

thyroid =pd.read_csv('./datasets/thyroidDF.csv')
display(thyroid)
thyroid.drop(['age', 'TT4', 'T4U', 'TSH', 'T3', 'FTI', 'TBG', 'referral_source', 'target', 'patient_id'], axis=1, inplace=True)

thyroid_map = {'F' : 1.0, 'M': 0.0, 'f': 0.0, 't': 1.0, 'NaN': 0.0}
thyroid = thyroid.replace(thyroid_map).to_numpy()

thyroid[np.argwhere(np.isnan(thyroid))] = 0.0

# Coreset Size Experiments

Here we evaluate the effect of coreset size on the Frobenius norm error. We generate 100 full rank binary matrices and compare the effect of coreset size on the average error of 10 runs of each algorithm. We run the experiment for values of $k \in \{2,3,5,10,15\}$.

In [ ]:
cs_low_rank_test = generate_noisy_rank_k(5000,50,0.5,5,0.0005)

In [ ]:
sizes = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
ks = [2, 3, 5, 10]
n_runs = 20

datasets = [cs_low_rank_test, congress, thyroid]
dataset_names = ["low_rank", "congress", "thyroid"]

coreset_dataframe = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Size': [],
    'Error': [],
    'Time': []
})
            
for k in ks:
    for (name, data) in zip(dataset_names, datasets):
        for i in range(n_runs):
            print(i)
            bmf = BMFKMeans_plus(k)
            res = bench_bmf(bmf=bmf, name="k-means bmf", k=k, data=data)
            
            for size in sizes: 
                row = pd.Series({'Dataset': name, 'Algorithm': 'bmf+', 'k': k, 'Size': size, 'Error': res[3], 'Time': res[2]})
                coreset_dataframe = pd.concat([coreset_dataframe, row.to_frame().T], ignore_index=True)

            for size in sizes:
                coreset_size = max(k, np.ceil(data.shape[1] * size).astype(int))
                
                bmf = BMFCoreset(k, coreset_size)
                res = bench_bmf(bmf=bmf, name="coreset bmf", k=k, data=data)
                
                row = pd.Series(
                    {'Dataset': name, 'Algorithm': 'coreset-bmf+', 'k': k, 'Size': size, 'Error': res[3], 'Time': res[2]})
                coreset_dataframe = pd.concat([coreset_dataframe, row.to_frame().T], ignore_index=True)
                
                bmf = BMFLWCoreset(k, coreset_size)
                res = bench_bmf(bmf=bmf, name="lw bmf", k=k, data=data)
                
                row = pd.Series(
                    {'Dataset': name, 'Algorithm': 'lw-coreset-bmf+', 'k': k, 'Size': size, 'Error': res[3], 'Time': res[2]})
                coreset_dataframe = pd.concat([coreset_dataframe, row.to_frame().T], ignore_index=True)

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid", palette="colorblind")

cs_df = coreset_dataframe

plot = sns.FacetGrid(data=cs_df[(cs_df['k'] == 5) | (cs_df['k'] == 10)], hue='Algorithm', col='Dataset', row='k', sharey=False, sharex=False)

plot.map(sns.lineplot, 'Size', 'Error', errorbar="se")

plot.add_legend(title="Algorithm")

fig = plot.fig
fig.savefig("coreset-plot.png", transparent=True, dpi=300)

In [ ]:
# coreset time 
sizes = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
ks = [2, 3, 5, 10]
n_runs = 20

datasets = [cs_low_rank_test, congress, thyroid]
dataset_names = ["low_rank", "congress", "thyroid"]

coreset_time = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Size': [],
    'Time': []
})

for k in ks:
    for (name, data) in zip(dataset_names, datasets):
        print(name)
        for i in range(n_runs):
            print(i)
            for size in sizes:
                coreset_size = max(k, np.ceil(data.shape[1] * size).astype(int))
                
                t0 = perf_counter()
                kmcs = KMeansCoreset(0.99, k).fit(data, coreset_size)
                res = perf_counter() - t0
                
                row = pd.Series(
                    {'Dataset': name, 'Algorithm': 'coreset', 'k': k, 'Size': size, 'Time': res})
                coreset_time = pd.concat([coreset_time, row.to_frame().T], ignore_index=True)
                
                t0 = perf_counter()
                kmcs = LWCoreset(coreset_size).fit(data)
                res = perf_counter() - t0
                
                row = pd.Series(
                    {'Dataset': name, 'Algorithm': 'lw-coreset', 'k': k, 'Size': size, 'Time': res})
                coreset_time = pd.concat([coreset_time, row.to_frame().T], ignore_index=True)

In [ ]:
cst_df = coreset_time

cst_df.groupby(['k', 'Algorithm'])['Time'].mean()

In [ ]:
cs_df.groupby(['k', 'Algorithm', 'Dataset']).mean()

In [ ]:
cs_df.to_pickle('./cs_df.pkl')

# Comparative Experiments on Real and Synthetic Data

In the experimental setup of these experiments, we compare only the quality (and runtime) of the approximation produced, using only the final matrix produced by the heuristics, without taking into account the low-rank factors produced by the algorithm and whether these are binary. Furthermore, the algorithms are used in a black box manner, which means that they may use the Boolean semiring, or GF(2) to perform matrix operations internally or may use non Boolean factors. 

We compare the frobenius norm error of the algorithms.

Even under ideal conditions, our post-processing step produces the best quality approximation with only a small runtime overhead for small k.

## Benchmarking on Synthetic Data

In [ ]:
synthetic_full = np.array([generate_synthetic_random(250, 50, 0.5) for i in range(25)])
synthetic_low_rank_5 = np.array([generate_synthetic_rank_k(250, 50, 0.5, 5) for i in range(25)])
synthetic_low_rank_10 = np.array([generate_synthetic_rank_k(250, 50, 0.5, 10) for i in range(25)])
synthetic_low_rank_15 = np.array([generate_synthetic_rank_k(250, 50, 0.5, 15) for i in range(25)])
synthetic_noisy_10_001 = np.array([generate_noisy_rank_k(250, 50, 0.5, 10, 0.01) for i in range(25)])
synthetic_noisy_10_0001 = np.array([generate_noisy_rank_k(250, 50, 0.5, 10, 0.001) for i in range(25)])
synthetic_full_ = np.array([generate_synthetic_random(250, 50, 0.1) for i in range(25)])
synthetic_low_rank_5_ = np.array([generate_synthetic_rank_k(250, 50, 0.1, 5) for i in range(25)])
synthetic_low_rank_10_ = np.array([generate_synthetic_rank_k(250, 50, 0.1, 10) for i in range(25)])
synthetic_low_rank_15_ = np.array([generate_synthetic_rank_k(250, 50, 0.1, 15) for i in range(25)])
synthetic_noisy_10_001_ = np.array([generate_noisy_rank_k(250, 50, 0.1, 10, 0.01) for i in range(25)])
synthetic_noisy_10_0001_ = np.array([generate_noisy_rank_k(250, 50, 0.1, 10, 0.001) for i in range(25)])


In [ ]:
ks = [2, 3, 5, 10, 15]
n_runs = 10

datasets = [synthetic_full, synthetic_low_rank_5, synthetic_low_rank_10, synthetic_low_rank_15, 
           synthetic_noisy_10_001, synthetic_noisy_10_0001,
            synthetic_full_, synthetic_low_rank_5_, synthetic_low_rank_10_, synthetic_low_rank_15_, 
           synthetic_noisy_10_001_, synthetic_noisy_10_0001_]
dataset_names = ["full", "lr5", "lr10", "lr15", "noisy10-001", "noisy10-0001", 
                 "full0.1", "lr5-0.1", "lr10-0.1", "lr15-0.1", "noisy10-001-0.1", "noisy10-0001-0.1", 
                ]

synth_df = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

            
for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        for i in range(n_runs):
            bmf = BMFKMeans(k)
            res = bench_bmf(bmf=bmf, name="bmf", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf', 'k': k, 'Error': res[3], 'Time': (res[2] / 1000)})
            synth_df = pd.concat([synth_df, row.to_frame().T], ignore_index=True)
            
            bmf = BMFKMeans_plus(k)
            res = bench_bmf(bmf=bmf, name="bmf+", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf+', 'k': k, 'Error': res[3], 'Time': (res[2]/1000)})
            synth_df = pd.concat([synth_df, row.to_frame().T], ignore_index=True)

In [ ]:
synth_df.to_pickle('./synth_df.pkl')

In [ ]:
import nimfa

ks = [2, 3, 5, 10, 15]
n_runs = 10

synth_df_nimfa = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print(k)
        for i in range(n_runs):
            avg_error = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()
                bmf = nimfa.Bmf(mat, seed="nndsvd", rank=k, max_iter=10000, lambda_w=1.1, lambda_h=1.1)
                bmf_fit = bmf()
                bmf_fitted = bmf.fitted()
                bmf_fitted = Binarizer(threshold=0.5).fit_transform(np.array(bmf_fitted))
                avg_time.append(perf_counter() - t0)
                avg_error.append(np.linalg.norm(bmf_fitted - mat))
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'nimfa', 'k': k, 'Error': np.average(avg_error), 'Time': np.average(avg_time)})
            synth_df_nimfa = pd.concat([synth_df_nimfa, row.to_frame().T], ignore_index=True)

In [ ]:
synth_df_nimfa.to_pickle('./synth_df_nimfa.pkl')

In [ ]:
from scipy.sparse import csr_matrix
from binmf import fit_spfact

ks = [2, 3, 5, 10, 15]
n_runs = 10

synth_df_binmf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print(k)
        for i in range(n_runs):
            avg_error = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()
                nrows, ncols = mat.shape

                sp_mat = csr_matrix(mat)
                row_fact = np.random.normal(size=(nrows, k))
                col_fact = np.random.normal(size=(ncols, k))
                fit_spfact(sp_mat, row_fact, col_fact, reg_param=1e-1, niter=200, nthreads=8) #adjust number of threads for your setup
                approx = row_fact @ col_fact.T
                avg_time.append(perf_counter() - t0)
                
                avg_error.append(np.linalg.norm(Binarizer(threshold=0.5).fit_transform(approx) - mat))
            
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'binmf', 'k': k, 'Error': np.average(avg_error), 'Time': np.average(avg_time)})
            
            synth_df_binmf = pd.concat([synth_df_binmf, row.to_frame().T], ignore_index=True)


In [ ]:
synth_df_binmf.to_pickle('./synth_df_binmf.pkl')

In [ ]:
# Reference: Ravanbakhsh Siamak, Poczos Barnabas, Greiner Russell, Boolean Matrix Factorization and Noisy Completion via Message Passing, ICML 2016

import numpy as np
import pdb

def log_ratio(x):
    return np.log(x) - np.log(1. - x)

def get_random_matrices(M, N, K, p_x_1 = .5, p_y_1 = .5, p_flip = 0, p_observe = .1):
    X = (np.random.rand(M, K) < p_x_1).astype(int)
    Y = (np.random.rand(N, K) < p_y_1).astype(int)
    Z = (X.dot(Y.T) > 0).astype(int)
    mask = np.random.rand(M,N) < p_observe
    O = Z.copy()
    
    flip = np.random.rand(M,N) < p_flip
    O[flip] = 1 - O[flip]
    mats = {'X':X, 'Y':Y, 'Z':Z, 'O':O, 'mask':mask}
    return mats

def hamming(X,Y):
    return np.sum(np.abs(X - Y) > 1e-5)

def density(X):
    return np.sum(X)/float(np.prod(X.shape))


def rec_error(Z, Zh):
    return np.sum(np.abs(Z - Zh))/float(np.prod(Z.shape))


def read_csv(fname, delimiter = ','):
    mat = np.genfromtxt(fname, delimiter=delimiter)
    return mat

In [ ]:
import numpy as np
import sys
import pdb


class MatrixCompletion(object):
    
    def __init__(self,
                 O, #observed matrix (only the parts indicated by mask will be used)
                 K, #hidden dim
                 mask = None,#boolean matrix the same size as O
                 min_sum = True,                 
                 tol = 1e-4,#tolerance for message updates
                 learning_rate = .2, #damping parameter
                 max_iter = 500, #maximum number of message passing updates
                 verbose = False,
                 p_x_1 = .5, #the prior probability of x=1. For regularization use small or large values in [0,1]
                 p_y_1 = .5, #the prior probability of y=1. For regularization use small or large values in [0,1]
                 #note that when p_x and p_y are uniform the MAP assignment is not sensitive
                 #to the following values, assuming they are the same and above .5
                 p_1_given_1 = .99, #the model of the noisy channel: probability of observing 1 for the input of 1
                 p_0_given_0 = .99, #similar to the above
                ):
        
        assert(p_x_1 < 1 and p_x_1 > 0)
        assert(p_y_1 < 1 and p_y_1 > 0)
        assert(p_1_given_1 > .5 and p_1_given_1 < 1)
        assert(p_0_given_0 > .5 and p_0_given_0 < 1)                
        
        self.O = O.astype(int)
        self.M,self.N = O.shape
        self.K = K
        self.verbose = verbose

        assert(self.K < min(self.M,self.N))
        if mask is not None:
            assert(mask.shape[0] == self.M and mask.shape[1] == self.N)
            self.mask = mask.astype(bool)
        else:
            self.mask = np.ones(O.shape, dtype=bool)
            
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.min_sum = min_sum
        self.num_edges = np.sum(self.mask)        

        self.update_adj_list()
        
        # will be used frequently
        self.pos_edges = np.nonzero(O[mask])[0]
        self.neg_edges = np.nonzero(1 - O[mask])[0]
        self.range_edges = np.arange(self.num_edges)
        self.cx = np.log(p_x_1) - np.log(1 - p_x_1)
        self.cy = np.log(p_y_1) - np.log(1 - p_y_1)
        self.co1 = np.log(p_1_given_1) - np.log(1. - p_0_given_0) #log(p(1|1)/p(1|0))
        self.co0 = np.log(1. - p_1_given_1) - np.log(p_0_given_0) ##log(p(0|1)/p(0|0))

    
    def init_msgs_n_marginals(self):
        self.marg_x = np.zeros((self.M, self.K))
        self.marg_y = np.zeros((self.N, self.K))
        self.in_x = np.zeros((self.num_edges, self.K)) #message going towards variable X: phi in the papger
        self.new_in_x = np.zeros((self.num_edges, self.K)) #the new one
        
        self.out_x = np.log((np.random.rand(self.num_edges, self.K)))#/self.M #message leaving variable x: phi_hat in the paper 
        self.in_y = np.zeros((self.num_edges, self.K)) #message leaving variable y: psi in the paper
        self.new_in_y = np.zeros((self.num_edges, self.K))
        self.out_y = np.log(np.random.rand(self.num_edges, self.K))#/self.N #psi_hat in the paper
        self.in_z = np.zeros((self.num_edges, self.K)) #gamma in the paper
        self.out_z = np.zeros((self.num_edges, self.K)) #gamma_hat in the paper
        
        
    def update_adj_list(self):
        ''' nbM: list of indices of nonzeros organized in rows
        nbM: list of indices of nonzeros organized in columns
        '''
        
        Mnz,Nnz = np.nonzero(self.mask)
        M = self.M
        N = self.N
        nbM = [[] for i in range(M)] 
        nbN = [[] for i in range(N)]

        for z in range(len(Mnz)):
            nbN[Nnz[z]].append(z)
            nbM[Mnz[z]].append(z)

        for i in range(M):
            nbM[i] = np.array(nbM[i], dtype=int)
        for i in range(N):
            nbN[i] = np.array(nbN[i], dtype=int)
            
        self.rows = nbM
        self.cols = nbN

        
    
    def run(self):
        self.init_msgs_n_marginals()
        iters = 1
        diff_msg = np.inf

        while (diff_msg > self.tol and iters <= self.max_iter) or iters < 5:
            self.update_min_sum()#(outX, outY, inZ, outZ, newInX, newInY, posEdges, negEdges,  opt)
            diff_msg = np.max(np.abs(self.new_in_x - self.in_x))
            self.in_x *= (1. - self.learning_rate)
            self.in_x += self.learning_rate * (self.new_in_x)
            self.in_y *= (1. - self.learning_rate)
            self.in_y += self.learning_rate * (self.new_in_y)
            self.update_margs()
            if self.verbose:
                print("iter %d, diff:%f" %(iters, diff_msg))
            #else:
                #print(iters)
                #sys.stdout.flush()
                
            iters += 1

        #recover X and Y from marginals and reconstruct Z
        self.X = (self.marg_x > 0).astype(int)
        self.Y = (self.marg_y > 0).astype(int)
        self.Z = (self.X.dot(self.Y.T) > 0).astype(int)

        
        
    def update_min_sum(self):
        self.in_z = np.minimum(np.minimum(self.out_x + self.out_y, self.out_x), self.out_y) #gamma update in the paper
        
        inz_pos = np.maximum(0.,self.in_z) # calculate it now, because we're chaning inz
        #find the second larges element along the 1st axis (there's also a 0nd! axis)
        inz_max_ind = np.argmax(self.in_z, axis=1)
        inz_max = np.maximum(-self.in_z[self.range_edges, inz_max_ind],0)
        self.in_z[self.range_edges, inz_max_ind] = -np.inf
        inz_max_sec = np.maximum(-np.max(self.in_z, axis=1),0) # update for gamma_hat in the paper
        sum_val = np.sum(inz_pos, axis=1)
        #penalties/rewards for confoming with observations
        sum_val[self.pos_edges] += self.co1
        sum_val[self.neg_edges] += self.co0
        
        tmp_inz_max = inz_max.copy()
        inz_pos =  sum_val[:, np.newaxis] - inz_pos
        
        for k in range(self.K):
            self_max_ind = np.nonzero(inz_max_ind == k)[0]#find the indices where the max incoming message is from k
            tmp_inz_max[self_max_ind] = inz_max_sec.take(self_max_ind)#replace the value of the max with the second largest value
            self.out_z[:, k] = np.minimum( tmp_inz_max, inz_pos[:,k])#see the update for gamma_hat
            tmp_inz_max[self_max_ind] = inz_max.take(self_max_ind)#fix tmp_iz_max for the next iter

        # update in_x and in_y: phi_hat and psi_hat in the paper
        self.new_in_x = np.maximum(self.out_z + self.out_y, 0) - np.maximum(self.out_y,0)
        self.new_in_y = np.maximum(self.out_z + self.out_x, 0) - np.maximum(self.out_x,0)

    

    def update_margs(self):
        #updates for phi and psi
        for m in range(self.M):
            self.marg_x[m,:] = np.sum(self.in_x.take(self.rows[m],axis=0), axis=0) + self.cx
            self.out_x[self.rows[m], :] = -self.in_x.take(self.rows[m],axis=0) + self.marg_x[m,:]

        for n in range(self.N):
            self.marg_y[n, :] = np.sum(self.in_y.take(self.cols[n], axis=0), axis=0) + self.cy
            self.out_y[self.cols[n], :] = -self.in_y.take(self.cols[n], axis=0) + self.marg_y[n,:]

In [ ]:
ks = [2, 3, 5, 10, 15]
n_runs = 10

synth_mpf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print(k)
        for i in range(n_runs):
            avg_error = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()             
                
                mask = np.random.rand(mat.shape[0], mat.shape[1]) < 0.99
                comp = MatrixCompletion(mat, k, mask = mask, min_sum = True, verbose = False, max_iter = 100)
                comp.run()
                
                avg_time.append(perf_counter() - t0)
                avg_error.append(np.linalg.norm(Binarizer(threshold=0.5).fit_transform(comp.Z) - mat))
            
            print(np.average(avg_error))
            print(np.average(avg_time))
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'mpbmf', 'k': k, 'Error': np.average(avg_error), 'Time': np.average(avg_time)})
            
            synth_mpf = pd.concat([synth_mpf, row.to_frame().T], ignore_index=True)


In [ ]:
synth_mpf.to_pickle("./synth_mpf.pkl")

In [ ]:
synth_df
synth_df_bmf = synth_df[(synth_df['Algorithm'] == 'bmf') | (synth_df['Algorithm'] == 'bmf+')]

display(synth_df_bmf)

synth_df_bmf.to_pickle("./synth_df_bmf.pkl")

In [ ]:
synth_df.groupby(["Dataset", "k", "Algorithm"]).mean()

## Benchmarking on Real Data

In [ ]:
ks = [2, 3, 5, 10, 15]
n_runs = 10

# datasets are generally three dimensional
datasets = [ orl, np.array([congress]), np.array([thyroid]) ]
dataset_names = ["orl", "congress", "thyroid",  
                ]

real_df_bmf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

            
for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        for i in range(n_runs):
            bmf = BMFKMeans(k)
            res = bench_bmf(bmf=bmf, name="bmf", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf', 'k': k, 'Error': res[3], 'Time': (res[2] / 1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)
            
            bmf = BMFKMeans_plus(k)
            res = bench_bmf(bmf=bmf, name="bmf+", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf+', 'k': k, 'Error': res[3], 'Time': (res[2]/1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)

In [ ]:
real_df_bmf.to_pickle('./real_df_bmf.pkl')

In [ ]:
import nimfa

ks = [2, 3, 5, 10, 15]
n_runs = 10

real_df_nimfa = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print(k)
        for i in range(n_runs):
            avg_error = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()
                bmf = nimfa.Bmf(mat, seed="nndsvd", rank=k, max_iter=10000, lambda_w=1.1, lambda_h=1.1)
                bmf_fit = bmf()
                bmf_fitted = bmf.fitted()
                bmf_fitted = Binarizer(threshold=0.5).fit_transform(np.array(bmf_fitted))
                avg_time.append(perf_counter() - t0)
                avg_error.append(np.linalg.norm(bmf_fitted - mat))
                print(bmf_fitted)
                print(mat)
                print(avg_error[-1])
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'nimfa', 'k': k, 'Error': np.average(avg_error), 'Time': np.average(avg_time)})
            real_df_nimfa = pd.concat([real_df_nimfa, row.to_frame().T], ignore_index=True)

In [ ]:
real_df_nimfa.to_pickle('./real_df_nimfa.pkl')

In [ ]:
from scipy.sparse import csr_matrix
from binmf import fit_spfact

ks = [2, 3, 5, 10, 15]
n_runs = 10

real_df_binmf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print(k)
        for i in range(n_runs):
            avg_error = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()
                nrows, ncols = mat.shape

                sp_mat = csr_matrix(mat)
                row_fact = np.random.normal(size=(nrows, k))
                col_fact = np.random.normal(size=(ncols, k))
                fit_spfact(sp_mat, row_fact, col_fact, reg_param=1e-1, niter=200, nthreads=8) #adjust number of threads for your setup
                approx = row_fact @ col_fact.T
                avg_time.append(perf_counter() - t0)
                
                avg_error.append(np.linalg.norm(Binarizer(threshold=0).fit_transform(approx) - mat))
            
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'binmf', 'k': k, 'Error': np.average(avg_error), 'Time': np.average(avg_time)})
            
            real_df_binmf = pd.concat([real_df_binmf, row.to_frame().T], ignore_index=True)


In [ ]:
real_df_binmf.to_pickle('./real_df_binmf.pkl')

In [ ]:
ks = [2, 3, 5, 10, 15]
n_runs = 10

real_df_mpf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print(k)
        for i in range(n_runs):
            avg_error = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()             
                
                mask = np.random.rand(mat.shape[0], mat.shape[1]) < 0.99
                comp = MatrixCompletion(mat, k, mask = mask, min_sum = True, verbose = False, max_iter = 100)
                comp.run()
                
                avg_time.append(perf_counter() - t0)
                avg_error.append(np.linalg.norm(Binarizer(threshold=0.5).fit_transform(comp.Z) - mat))
            
            print(np.average(avg_error))
            print(np.average(avg_time))
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'mpbmf', 'k': k, 'Error': np.average(avg_error), 'Time': np.average(avg_time)})
            
            real_df_mpf = pd.concat([real_df_mpf, row.to_frame().T], ignore_index=True)


In [ ]:
real_df_mpf.to_pickle('./real_df_mpf.pkl')

# Comparisons on Real Data for different lp-norms and regular matrix multiplication

We now compare the approximations produced by the heuristics for Lp norms on the integers for $p \in \{1, 2, 5\}$. Note that the case $p = 2$ is the setting of minimizing the Frobenius norm that we initially discuss in our paper. 

As some of the heuristics do not produce low rank factors that are boolean, we binarize them using a threshold chosen such that the error across experiments is minimized. This is the case for the NIMFA and binmf heuristics.


In [ ]:
ks = [2, 3, 5, 10, 15]
n_runs = 10

# datasets are generally three dimensional
datasets = [ orl, np.array([congress]), np.array([thyroid]) ]
dataset_names = ["orl", "congress", "thyroid",  
                ]

real_df_bmf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

            
for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        for i in range(n_runs):
            bmf = BMFKMeans(k)
            res = bench_bmf(bmf=bmf, name="bmf", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf', 'k': k, 'Error': res[3], 'Time': (res[2] / 1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)
            
            bmf = BMFKMeans_plus_int_1(k)
            res = bench_bmf(bmf=bmf, name="bmf+ (int, l1)", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf+ (int, l1)', 'k': k, 'Error': res[3], 'Time': (res[2]/1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)
            
            bmf = BMFKMeans_plus_int_2(k)
            res = bench_bmf(bmf=bmf, name="bmf+ (int, l2)", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf+ (int, l2)', 'k': k, 'Error': res[3], 'Time': (res[2]/1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)
            
            bmf = BMFKMeans_plus_int_5(k)
            res = bench_bmf(bmf=bmf, name="bmf+ (int, l5)", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf+ (int, l5)', 'k': k, 'Error': res[3], 'Time': (res[2]/1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)
            
            

In [ ]:
real_df_bmf.to_pickle('./real_df_bmf_int.pkl')

In [ ]:
bmf = nimfa.Bmf(congress, seed="nndsvd", rank=15, max_iter=10000, lambda_w=1.1, lambda_h=1.1)
bmf_fit = bmf()
b = Binarizer(threshold=0.3)               
l = b.fit_transform(np.asarray(bmf.basis()))
r = b.fit_transform(np.asarray(bmf.coef()))
#l = bmf.basis()
#r = bmf.coef()
bmf_fitted = l @ r

print(np.square(np.linalg.norm(congress - bmf_fitted)))

plt.imshow(bmf_fitted)
print(l)

In [ ]:
import nimfa

ks = [2, 3, 5, 10, 15]
n_runs = 10

real_df_nimfa = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print(k)
        for i in range(n_runs):
            avg_error_l1 = []
            avg_error_l2 = []
            avg_error_l5 = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()
                bmf = nimfa.Bmf(mat, seed="nndsvd", rank=k, max_iter=10000, lambda_w=1.1, lambda_h=1.1)
                bmf_fit = bmf()
                b = Binarizer(threshold=0.3)               
                l = b.fit_transform(np.asarray(bmf.basis()))
                r = b.fit_transform(np.asarray(bmf.coef()))
                bmf_fitted = l @ r

                #bmf_fitted = bmf.fitted()
                #bmf_fitted = Binarizer(threshold=0.5).fit_transform(np.array(bmf_fitted))
                avg_time.append(perf_counter() - t0)
                avg_error_l2.append(np.square(np.linalg.norm(bmf_fitted - mat)))
                avg_error_l1.append(np.sum(np.abs(bmf_fitted - mat)))
                avg_error_l5.append(np.power(np.linalg.norm((bmf_fitted-mat).flatten(), ord=5) , 5))
                #print(bmf_fitted)
                #print(mat)
                #print(avg_error[-1])
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'nimfa l1', 'k': k, 'Error': np.average(avg_error_l1), 'Time': np.average(avg_time)})
            real_df_nimfa = pd.concat([real_df_nimfa, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'nimfa l2', 'k': k, 'Error': np.average(avg_error_l2), 'Time': np.average(avg_time)})
            real_df_nimfa = pd.concat([real_df_nimfa, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'nimfa l5', 'k': k, 'Error': np.average(avg_error_l5), 'Time': np.average(avg_time)})
            real_df_nimfa = pd.concat([real_df_nimfa, row.to_frame().T], ignore_index=True)

In [ ]:
real_df_nimfa.to_pickle('./real_df_nimfa_int.pkl')

In [ ]:
from scipy.sparse import csr_matrix
from binmf import fit_spfact

mat = orl[6]
k = 15

nrows, ncols = mat.shape

sp_mat = csr_matrix(mat)
row_fact = np.random.normal(size=(nrows, k))
col_fact = np.random.normal(size=(ncols, k))
fit_spfact(sp_mat, row_fact, col_fact, reg_param=1e-1, niter=200, nthreads=8) #adjust number of threads for your setup
b = Binarizer(threshold=0.6)
approx = b.fit_transform(row_fact) @ b.fit_transform(col_fact.T)

print(np.square(np.linalg.norm(mat - approx)))

plt.imshow(approx)
print(approx)

In [ ]:
from scipy.sparse import csr_matrix
from binmf import fit_spfact

ks = [2, 3, 5, 10, 15]
n_runs = 10

real_df_binmf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print("k: %s" % k)
        for i in range(n_runs):
            print(i)
            j=0
            avg_error_l1 = []
            avg_error_l2 = []
            avg_error_l5 = []
            avg_time = []
            for mat in data:
                if j % 50 == 0:
                    print(j) 
                j+=1
                
                t0 = perf_counter()
                nrows, ncols = mat.shape

                sp_mat = csr_matrix(mat)
                b = Binarizer(threshold=0.6)
                row_fact = np.random.normal(size=(nrows, k))
                col_fact = np.random.normal(size=(ncols, k))
                fit_spfact(sp_mat, row_fact, col_fact, reg_param=1e-1, niter=200, nthreads=8) #adjust number of threads for your setup
                avg_time.append(perf_counter() - t0)

                bmf_fitted = b.fit_transform(row_fact) @ b.fit_transform(col_fact.T)
                
                avg_error_l2.append(np.square(np.linalg.norm(bmf_fitted - mat)))
                avg_error_l1.append(np.sum(np.abs(bmf_fitted - mat)))
                avg_error_l5.append(np.power(np.linalg.norm((bmf_fitted-mat).flatten(), ord=5) , 5))
            
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'binmf l1', 'k': k, 'Error': np.average(avg_error_l1), 'Time': np.average(avg_time)})
            real_df_binmf = pd.concat([real_df_binmf, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'binmf l2', 'k': k, 'Error': np.average(avg_error_l2), 'Time': np.average(avg_time)})
            real_df_binmf = pd.concat([real_df_binmf, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'binmf l5', 'k': k, 'Error': np.average(avg_error_l5), 'Time': np.average(avg_time)})
            real_df_binmf = pd.concat([real_df_binmf, row.to_frame().T], ignore_index=True)


In [ ]:
real_df_binmf.to_pickle('./real_df_binmf_int.pkl')

In [ ]:
mat = orl[0]
k = 15

mask = np.random.rand(mat.shape[0], mat.shape[1]) < 0.99
comp = MatrixCompletion(mat, k, mask = mask, min_sum = True, verbose = False, max_iter = 100)
comp.run()

print(comp.Y)
print(comp.X)
plt.imshow(comp.X @ comp.Y.T)


In [ ]:
ks = [2, 3, 5, 10, 15]
n_runs = 10

real_df_mpf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print('k: %s' % k)
        for i in range(n_runs):
            print(i)
            avg_error_l1 = []
            avg_error_l2 = []
            avg_error_l5 = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()             
                
                mask = np.random.rand(mat.shape[0], mat.shape[1]) < 0.99
                comp = MatrixCompletion(mat, k, mask = mask, min_sum = True, verbose = False, max_iter = 100)
                comp.run()
                avg_time.append(perf_counter() - t0)
                bmf_fitted = comp.X @ comp.Y.T
                
                #avg_error.append(np.linalg.norm(Binarizer(threshold=0.5).fit_transform(comp.Z) - mat))
                avg_error_l2.append(np.square(np.linalg.norm(bmf_fitted - mat)))
                avg_error_l1.append(np.sum(np.abs(bmf_fitted - mat)))
                avg_error_l5.append(np.power(np.linalg.norm((bmf_fitted-mat).flatten(), ord=5) , 5))

           
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'mpbmf l1', 'k': k, 'Error': np.average(avg_error_l1), 'Time': np.average(avg_time)})
            real_df_mpf = pd.concat([real_df_mpf, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'mpbmf l2', 'k': k, 'Error': np.average(avg_error_l2), 'Time': np.average(avg_time)})
            real_df_mpf = pd.concat([real_df_mpf, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'mpbmf l5', 'k': k, 'Error': np.average(avg_error_l5), 'Time': np.average(avg_time)})
            real_df_mpf = pd.concat([real_df_mpf, row.to_frame().T], ignore_index=True)


In [ ]:
real_df_mpf.to_pickle('./real_df_mpf_int.pkl')

# Int with Synthetic

In this section we perform experiments where the matrix multiplication is performed in the standard way over the reals, rather than the boolean semiring or GF(2). This is especially bad for existing heuristics, as they internally are usually tuned to the Boolean semiring. The NIMFA and binmf heuristics do not have a guarantee on producing 0-1 valued low rank factors U and V, so we apply a best-effort thresholding, to produce these factors.

*We note again here, that the existing heuristics are not tuned to this setting and therefore we do not expect them to perform well*. The heuristic of Kumar et. al and our extension that guesses a good U V pair based on centers obtained from the coreset does 

In [ ]:
ks = [2, 3, 5, 10, 15]
n_runs = 10

# datasets are generally three dimensional
datasets = [synthetic_full, synthetic_low_rank_5, synthetic_low_rank_10, synthetic_low_rank_15, 
           synthetic_noisy_10_001, synthetic_noisy_10_0001,
            synthetic_full_, synthetic_low_rank_5_, synthetic_low_rank_10_, synthetic_low_rank_15_, 
           synthetic_noisy_10_001_, synthetic_noisy_10_0001_]
dataset_names = ["full", "lr5", "lr10", "lr15", "noisy10-001", "noisy10-0001", 
                 "full0.1", "lr5-0.1", "lr10-0.1", "lr15-0.1", "noisy10-001-0.1", "noisy10-0001-0.1", 
                ]

real_df_bmf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

            
for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        for i in range(n_runs):
            bmf = BMFKMeans(k)
            res = bench_bmf(bmf=bmf, name="bmf", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf', 'k': k, 'Error': res[3], 'Time': (res[2] / 1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)
            
            bmf = BMFKMeans_plus_int_1(k)
            res = bench_bmf(bmf=bmf, name="bmf+ (int, l1)", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf+ (int, l1)', 'k': k, 'Error': res[3], 'Time': (res[2]/1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)
            
            bmf = BMFKMeans_plus_int_2(k)
            res = bench_bmf(bmf=bmf, name="bmf+ (int, l2)", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf+ (int, l2)', 'k': k, 'Error': res[3], 'Time': (res[2]/1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)
            
            bmf = BMFKMeans_plus_int_5(k)
            res = bench_bmf(bmf=bmf, name="bmf+ (int, l5)", k=k, data=data)
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'bmf+ (int, l5)', 'k': k, 'Error': res[3], 'Time': (res[2]/1000)})
            real_df_bmf = pd.concat([real_df_bmf, row.to_frame().T], ignore_index=True)
            
            

In [ ]:
real_df_bmf.to_pickle('./synth_df_bmf_int.pkl')

In [ ]:
import nimfa

ks = [2, 3, 5, 10, 15]
n_runs = 10

real_df_nimfa = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print(k)
        for i in range(n_runs):
            avg_error_l1 = []
            avg_error_l2 = []
            avg_error_l5 = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()
                bmf = nimfa.Bmf(mat, seed="nndsvd", rank=k, max_iter=10000, lambda_w=1.1, lambda_h=1.1)
                bmf_fit = bmf()
                b = Binarizer(threshold=0.3)               
                l = b.fit_transform(np.asarray(bmf.basis()))
                r = b.fit_transform(np.asarray(bmf.coef()))
                bmf_fitted = l @ r

                #bmf_fitted = bmf.fitted()
                #bmf_fitted = Binarizer(threshold=0.5).fit_transform(np.array(bmf_fitted))
                avg_time.append(perf_counter() - t0)
                avg_error_l2.append(np.square(np.linalg.norm(bmf_fitted - mat)))
                avg_error_l1.append(np.sum(np.abs(bmf_fitted - mat)))
                avg_error_l5.append(np.power(np.linalg.norm((bmf_fitted-mat).flatten(), ord=5) , 5))
                #print(bmf_fitted)
                #print(mat)
                #print(avg_error[-1])
                
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'nimfa l1', 'k': k, 'Error': np.average(avg_error_l1), 'Time': np.average(avg_time)})
            real_df_nimfa = pd.concat([real_df_nimfa, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'nimfa l2', 'k': k, 'Error': np.average(avg_error_l2), 'Time': np.average(avg_time)})
            real_df_nimfa = pd.concat([real_df_nimfa, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'nimfa l5', 'k': k, 'Error': np.average(avg_error_l5), 'Time': np.average(avg_time)})
            real_df_nimfa = pd.concat([real_df_nimfa, row.to_frame().T], ignore_index=True)

In [ ]:
real_df_nimfa.to_pickle('./synth_df_nimfa_int.pkl')

In [ ]:
from scipy.sparse import csr_matrix
from binmf import fit_spfact

ks = [2, 3, 5, 10, 15]
n_runs = 10

real_df_binmf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print("k: %s" % k)
        for i in range(n_runs):
            print(i)
            avg_error_l1 = []
            avg_error_l2 = []
            avg_error_l5 = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()
                nrows, ncols = mat.shape

                sp_mat = csr_matrix(mat)
                b = Binarizer(threshold=0.6)
                row_fact = np.random.normal(size=(nrows, k))
                col_fact = np.random.normal(size=(ncols, k))
                fit_spfact(sp_mat, row_fact, col_fact, reg_param=1e-1, niter=200, nthreads=8) #adjust number of threads for your setup
                avg_time.append(perf_counter() - t0)

                bmf_fitted = b.fit_transform(row_fact) @ b.fit_transform(col_fact.T)
                
                avg_error_l2.append(np.square(np.linalg.norm(bmf_fitted - mat)))
                avg_error_l1.append(np.sum(np.abs(bmf_fitted - mat)))
                avg_error_l5.append(np.power(np.linalg.norm((bmf_fitted-mat).flatten(), ord=5) , 5))
            
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'binmf l1', 'k': k, 'Error': np.average(avg_error_l1), 'Time': np.average(avg_time)})
            real_df_binmf = pd.concat([real_df_binmf, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'binmf l2', 'k': k, 'Error': np.average(avg_error_l2), 'Time': np.average(avg_time)})
            real_df_binmf = pd.concat([real_df_binmf, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'binmf l5', 'k': k, 'Error': np.average(avg_error_l5), 'Time': np.average(avg_time)})
            real_df_binmf = pd.concat([real_df_binmf, row.to_frame().T], ignore_index=True)


In [ ]:
real_df_binmf.to_pickle('./synth_df_binmf_int.pkl')

In [ ]:
ks = [2, 3, 5, 10, 15]
n_runs = 10

real_df_mpf = pd.DataFrame.from_dict({
    'Dataset': [],
    'Algorithm': [],
    'k': [],
    'Error': [],
    'Time': []
})

for (name, data) in zip(dataset_names, datasets):
    print(name)
    for k in ks:
        print('k: %s' % k)
        for i in range(n_runs):
            print(i)
            avg_error_l1 = []
            avg_error_l2 = []
            avg_error_l5 = []
            avg_time = []
            for mat in data:
                t0 = perf_counter()             
                
                mask = np.random.rand(mat.shape[0], mat.shape[1]) < 0.99
                comp = MatrixCompletion(mat, k, mask = mask, min_sum = True, verbose = False, max_iter = 100)
                comp.run()
                avg_time.append(perf_counter() - t0)
                bmf_fitted = comp.X @ comp.Y.T
                
                #avg_error.append(np.linalg.norm(Binarizer(threshold=0.5).fit_transform(comp.Z) - mat))
                avg_error_l2.append(np.square(np.linalg.norm(bmf_fitted - mat)))
                avg_error_l1.append(np.sum(np.abs(bmf_fitted - mat)))
                avg_error_l5.append(np.power(np.linalg.norm((bmf_fitted-mat).flatten(), ord=5) , 5))

           
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'mpbmf l1', 'k': k, 'Error': np.average(avg_error_l1), 'Time': np.average(avg_time)})
            real_df_mpf = pd.concat([real_df_mpf, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'mpbmf l2', 'k': k, 'Error': np.average(avg_error_l2), 'Time': np.average(avg_time)})
            real_df_mpf = pd.concat([real_df_mpf, row.to_frame().T], ignore_index=True)
            row = pd.Series(
                {'Dataset': name, 'Algorithm': 'mpbmf l5', 'k': k, 'Error': np.average(avg_error_l5), 'Time': np.average(avg_time)})
            real_df_mpf = pd.concat([real_df_mpf, row.to_frame().T], ignore_index=True)


In [ ]:
real_df_mpf.to_pickle('./synth_df_mpf_int.pkl')

# Data Analysis

We first analyze the outcomes of the "steelmanned" heuristic comparison, where each heuristic is used as detailed by the implementers.

In [425]:
# load real data
real_df_bmf = pd.read_pickle('./real_df_bmf.pkl')
real_df_binmf = pd.read_pickle('./real_df_binmf.pkl')
real_df_mpf = pd.read_pickle('./real_df_mpf.pkl')
real_df_nimfa = pd.read_pickle('./real_df_nimfa.pkl')

real_df = pd.concat([real_df_bmf, real_df_mpf, real_df_nimfa, real_df_binmf], ignore_index=True)

# load synthetic data
synth_df_bmf = pd.read_pickle('./synth_df_bmf.pkl')
synth_df_binmf = pd.read_pickle('./synth_df_binmf.pkl')
synth_df_mpf = pd.read_pickle('./synth_mpf.pkl')
synth_df_nimfa = pd.read_pickle('./synth_df_nimfa.pkl')

synth_df = pd.concat([synth_df_bmf, synth_df_binmf, synth_df_mpf, synth_df_nimfa])


## Results for Real Data

Here we show the Frobenius norm error and Runtime (in seconds) for each algorithm on real datasets.

In [426]:
grouped_df = real_df.groupby(['Dataset', 'k', 'Algorithm']).mean()
grouped_df.unstack()

Error                                                   Time  \
Algorithm        binmf         bmf       bmf+      mpbmf      nimfa     binmf   
Dataset  k                                                                      
congress 2   36.458482   40.037482  38.820098  38.809791  36.359318  0.090348   
         3   32.512418   38.363051  36.624927  35.889981  32.710854  0.102086   
         5   24.286930   35.696393  32.712793  31.141360  27.748874  0.119905   
         10   5.899723   32.730459  23.919929  22.471524  18.439089  0.102877   
         15   0.000000   30.929173  14.807827  15.469016   9.591663  0.089422   
orl      2   33.713419   39.376678  37.785530  35.857652  33.521305  0.090572   
         3   28.626366   35.716104  34.569188  32.237612  29.711777  0.088825   
         5   22.105650   31.672486  30.712163  27.680846  25.577315  0.084834   
         10  12.099023   26.449686  25.714423  21.628539  21.382890  0.081724   
         15   5.205162   23.414144  22.794089  17.845095  19.691523  0.081194   
thyroid  2   95.822162  106.600675  98.622961  90.474314  91.580566  0.376068   
         3   84.365356   94.480695  90.472925  75.475512  73.925638  0.312943   
         5   50.023772   82.704286  80.395914  78.513723  61.846584  0.398811   
         10  23.591007   66.032060  55.419848  54.045930  52.933921  0.346888   
         15   3.571106   57.552221  38.903848  39.232925  46.711883  0.348516   

                                                      
Algorithm         bmf      bmf+      mpbmf     nimfa  
Dataset  k                                            
congress 2   0.001975  0.003310   0.280718  0.006909  
         3   0.002328  0.004103   0.311166  0.013638  
         5   0.004624  0.005205   0.332855  0.016217  
         10  0.003197  0.016910   0.407083  0.022573  
         15  0.007394  0.246655   0.480527  0.027471  
orl      2   0.001990  0.002936   0.203744  0.011625  
         3   0.002908  0.004662   0.241613  0.013129  
         5   0.003809  0.005817   0.289372  0.015375  
         10  0.004258  0.022310   0.415707  0.019140  
         15  0.006068  0.318042   0.575532  0.022177  
thyroid  2   0.012622  0.014159   7.063550  0.044308  
         3   0.014406  0.018722   7.822042  0.092907  
         5   0.031802  0.025212   8.860249  0.132118  
         10  0.028935  0.059646  12.686316  0.241407  
         15  0.026658  0.313364  16.237675  0.432735

## Results for Synthetic Data

Here we show the Frobenius norm error and Runtime (in seconds) for each algorithm on synthetic datasets.

In [427]:
grouped_df = synth_df.groupby(['Dataset', 'k', 'Algorithm']).mean()
grouped_df.unstack()

Error                                              \
Algorithm                binmf        bmf       bmf+      mpbmf      nimfa   
Dataset          k                                                           
full             2   73.344298  75.750915  72.333028  71.256986  71.321495   
                 3   71.831318  74.280728  69.928525  69.385181  68.674021   
                 5   69.252822  72.157767  65.764912  66.563674  64.856944   
                 10  62.785281  68.657704  57.353965  61.535130  58.478444   
                 15  56.163514  66.404657  50.373358  57.918306  53.723584   
full0.1          2   43.009379  36.000430  35.010907  34.946703  35.164001   
                 3   42.409356  35.899464  34.884510  34.948634  34.976351   
                 5   39.331131  35.637114  34.608827  35.530870  34.223749   
                 10  34.073479  35.023489  33.868040  35.812739  31.749037   
                 15  29.300266  34.317619  33.025709  38.451906  28.995963   
lr10             2   73.169735  75.817642  72.161664  71.090447  71.680565   
                 3   71.561317  74.320028  69.554721  69.135761  68.961825   
                 5   68.815887  72.048301  64.746181  66.119482  64.841498   
                 10  61.768702  68.193327  28.368406  60.222202  57.948659   
                 15  55.343812  65.645340   0.799573  56.017076  52.942195   
lr10-0.1         2   32.765260  30.820627  30.521675  27.625313  28.510833   
                 3   28.609959  28.506134  28.131885  25.238997  25.479182   
                 5   17.199439  24.734379  23.198115  20.365842  19.927415   
                 10   2.605244  18.313160  10.201085   7.566958   8.757867   
                 15   0.196752  15.221658   2.504529   4.691869   5.410569   
lr15             2   73.461105  75.708405  72.273592  71.187829  71.268859   
                 3   71.885052  74.246647  69.898775  69.338224  68.712767   
                 5   69.421449  72.067391  65.710864  66.565762  64.809208   
                 10  62.643136  68.609598  56.540031  61.474079  58.391770   
                 15  56.199033  66.368334  29.212435  57.732194  53.589677   
lr15-0.1         2   39.467855  38.697035  38.229198  35.638083  36.530253   
                 3   37.376478  37.087185  36.190913  33.650332  34.211802   
                 5   30.334947  33.716537  32.177006  29.832138  29.483310   
                 10  13.518119  28.122054  22.340419  20.330422  19.781748   
                 15   3.752210  25.274851  14.232960  11.597783  13.381887   
lr5              2   66.111364  72.472089  67.098743  66.013343  67.823084   
                 3   61.581214  69.161153  60.017709  62.280223  64.038011   
                 5   53.020314  63.956862  26.933413  55.245308  56.724761   
                 10  34.338310  52.869364   0.693289  41.015698  42.546584   
                 15  19.072401  43.320731   0.000000  32.752151  31.133041   
lr5-0.1          2   20.671033  20.506771  20.369701  16.458187  15.809840   
                 3   12.877655  16.982355  16.581775  13.068700  11.961674   
                 5    2.529595  11.121688   8.431304   4.618262   5.122947   
                 10   0.000000   5.149004   0.000000   0.723558   2.347018   
                 15   0.000000   1.519957   0.000000   0.383233   1.447927   
noisy10-0001     2   73.371909  75.830112  72.314566  71.212894  71.573618   
                 3   71.633068  74.332280  69.618142  69.270222  69.007748   
                 5   68.985594  72.054578  64.695129  66.218066  64.964312   
                 10  62.120176  68.194114  33.776000  60.342012  58.077464   
                 15  55.412943  65.566467   4.821331  56.164721  53.212566   
noisy10-0001-0.1 2   33.303616  32.482432  32.142283  29.307728  29.963531   
                 3   29.460065  30.046449  29.523761  26.942610  27.099691   
                 5   19.396051  26.152327  24.632310  21.993109  21.320073   
                 10   5.687233  19.784124  12.0

# Results for $L_p$ Norms and Standard Matrix Product

We now showcase the results of $L_p$ loss for the approximations produced by different heuristics.

Runtimes have been omitted, as they are the same as in the case of the above experiments.

In [413]:
# load real data
real_df_bmf = pd.read_pickle('./real_df_bmf_int.pkl')
real_df_binmf = pd.read_pickle('./real_df_binmf_int.pkl')
real_df_mpf = pd.read_pickle('./real_df_mpf_int.pkl')
real_df_nimfa = pd.read_pickle('./real_df_nimfa_int.pkl')

real_df = pd.concat([real_df_bmf, real_df_mpf, real_df_nimfa, real_df_binmf], ignore_index=True)
real_l1 = real_df.loc[real_df['Algorithm'].isin(['binmf l1', 'bmf', 'bmf+ (int, l1)', 'mpbmf l1', 'nimfa l1'])]
real_l2 = real_df.loc[real_df['Algorithm'].isin(['binmf l2', 'bmf', 'bmf+ (int, l2)', 'mpbmf l2', 'nimfa l2'])]
real_l5 = real_df.loc[real_df['Algorithm'].isin(['binmf l5', 'bmf', 'bmf+ (int, l5)', 'mpbmf l5', 'nimfa l5'])]


# load synthetic data
synth_df_bmf = pd.read_pickle('./synth_df_bmf_int.pkl')
synth_df_binmf = pd.read_pickle('./synth_df_binmf_int.pkl')
synth_df_mpf = pd.read_pickle('./synth_df_mpf_int.pkl')
synth_df_nimfa = pd.read_pickle('./synth_df_nimfa_int.pkl')

synth_df = pd.concat([synth_df_bmf, synth_df_binmf, synth_df_mpf, synth_df_nimfa])

synth_l1 = synth_df.loc[synth_df['Algorithm'].isin(['binmf l1', 'bmf', 'bmf+ (int, l1)', 'mpbmf l1', 'nimfa l1'])]
synth_l2 = synth_df.loc[synth_df['Algorithm'].isin(['binmf l2', 'bmf', 'bmf+ (int, l2)', 'mpbmf l2', 'nimfa l2'])]
synth_l5 = synth_df.loc[synth_df['Algorithm'].isin(['binmf l5', 'bmf', 'bmf+ (int, l5)', 'mpbmf l5', 'nimfa l5'])]


In [414]:
grouped_df = real_l1.groupby(['Dataset', 'k', 'Algorithm']).mean()
grouped_df['Error'].unstack()

Algorithm       binmf l1          bmf  bmf+ (int, l1)     mpbmf l1    nimfa l1
Dataset  k                                                                    
congress 2    2535.40000   1603.00000      1522.00000   1522.70000   2011.0000
         3    2723.50000   1466.60000      1378.90000   1538.40000   2045.0000
         5    2614.10000   1267.00000      1167.80000   1763.70000   2264.0000
         10   3005.40000   1049.20000       969.90000   1876.80000   1102.0000
         15   2711.40000    920.40000       872.30000   1677.20000    380.0000
orl      2    4525.91425   1585.28850      1473.15475   3487.70375   2072.6775
         3    4608.26300   1311.40900      1250.40375   4514.47350   3286.6725
         5    4586.95800   1032.48625       997.73050   5925.21800   4545.2375
         10   5081.62300    722.43050       704.79500   8377.60575   3898.6275
         15   5351.15800    568.44675       556.85200  10488.95875   3813.2350
thyroid  2   32295.60000  12094.60000     10187.90000  19573.60000   8225.0000
         3   48312.90000   9302.70000      9032.90000  26269.90000  17151.0000
         5   47199.10000   6931.10000      6987.60000  18783.50000  46439.0000
         10  42135.60000   4298.70000      4343.30000  24350.70000  19392.0000
         15  43816.30000   3422.30000      3237.00000  27547.90000  11561.0000

In [428]:
grouped_df = real_l2.groupby(['Dataset', 'k', 'Algorithm']).mean()
grouped_df['Error'] = grouped_df['Error'].apply(lambda x: np.sqrt(x))
grouped_df['Error'].unstack()

Algorithm      binmf l2         bmf  bmf+ (int, l2)    mpbmf l2    nimfa l2
Dataset  k                                                                 
congress 2    50.374597   40.037482       39.064050   39.111379   44.844175
         3    52.332590   38.296214       37.202150   40.129789   47.717921
         5    51.612983   35.594943       34.461573   45.553266   49.699095
         10   57.956881   32.391357       31.115912   50.823223   33.376639
         15   52.429000   30.338095       29.337689   50.888113   20.396078
orl      2    67.330998   39.815682       38.433442   61.410758   46.191368
         3    68.105114   36.213382       35.349707   78.544004   59.202006
         5    68.738039   32.132324       31.609785  101.445172   72.031955
         10   75.935970   26.878067       26.564455  146.238995   66.185856
         15   79.559148   23.842121       23.613534  186.631093   63.444030
thyroid  2   179.715887  109.975452      100.409661  142.967829   90.691786
         3   220.107474   96.450505       90.637740  177.972751  132.034087
         5   220.003864   83.253228       81.855971  149.351599  248.991968
         10  206.860339   65.564472       65.139082  177.327663  139.255161
         15  209.714330   58.500427       57.562140  197.155522  107.522091

In [429]:
grouped_df = real_l5.groupby(['Dataset', 'k', 'Algorithm']).mean()
grouped_df['Error'] = grouped_df['Error'].apply(lambda x: np.power(x, 1.0 / 5.0))
grouped_df['Error'].unstack()

Algorithm    binmf l5       bmf  bmf+ (int, l5)   mpbmf l5   nimfa l5
Dataset  k                                                           
congress 2   4.807647  4.375087        4.332224   4.388488   4.578070
         3   4.943204  4.297960        4.250074   4.826221   5.603591
         5   5.100251  4.174028        4.116247   5.923003   5.568467
         10  6.406303  4.019497        3.944662   6.985513   4.183864
         15  5.069224  3.915573        3.861609   7.601854   3.915233
orl      2   5.411126  4.365376        4.303755   5.994646   4.955089
         3   5.506552  4.202886        4.162835   8.038793   5.812394
         5   5.864436  4.006608        3.980946  11.232400   6.842898
         10  7.244714  3.730433        3.713458  18.873718   6.533988
         15  7.774064  3.555805        3.541660  26.913122   5.919185
thyroid  2   7.978429  6.554179        6.311074   7.990142   6.067743
         3   8.716929  6.219018        6.157791  10.264366   7.345233
         5   9.393117  5.863547        5.871882   9.686046  12.284920
         10  8.801714  5.329266        5.354880  10.939349   7.203184
         15  8.575015  5.091709        5.058701  12.225846   6.495298

In [417]:
grouped_df = synth_l1.groupby(['Dataset', 'k', 'Algorithm']).mean()
grouped_df['Error'].unstack()

Algorithm            binmf l1       bmf  bmf+ (int, l1)  mpbmf l1  nimfa l1
Dataset          k                                                         
full             2   5894.808  5736.392        5383.992  5590.284   6056.24
                 3   5841.592  5516.160        5204.740  5719.600   6183.36
                 5   5799.004  5205.024        4959.436  5779.136   6418.88
                 10  5637.484  4712.456        4578.772  5550.372   6107.28
                 15  5516.280  4406.472        4323.488  5314.516   5569.40
full0.1          2   1934.784  1289.460        1227.364  1225.932   1499.20
                 3   2135.224  1282.024        1221.808  1230.428   1744.28
                 5   2228.592  1265.340        1202.316  1266.888   1869.32
                 10  1738.056  1221.160        1153.768  1353.024   1535.76
                 15  1804.804  1176.524        1100.732  1646.632   1234.60
lr10             2   5869.036  5741.388        5357.288  5590.468   5964.96
                 3   5825.476  5517.924        5166.236  5748.572   6182.28
                 5   5782.580  5197.728        4898.900  5782.228   5966.28
                 10  5620.568  4647.344        4476.280  5617.380   5957.00
                 15  5504.988  4295.128        4179.220  5388.244   5641.16
lr10-0.1         2   1371.160  1029.800        1004.948   837.400   1235.84
                 3   1526.156   890.364         856.028   715.292   1145.28
                 5   1794.068   667.928         613.124   488.084    695.64
                 10  1738.072   372.532         227.932   130.588    186.96
                 15  1180.384   257.132         108.320    99.376    139.64
lr15             2   5868.216  5732.744        5375.496  5563.124   6036.00
                 3   5823.016  5515.080        5197.868  5688.616   6248.72
                 5   5784.820  5207.972        4952.756  5732.756   6429.76
                 10  5633.284  4710.444        4569.272  5526.260   6235.68
                 15  5502.188  4401.692        4318.540  5283.496   5712.28
lr15-0.1         2   1753.676  1550.032        1506.412  1322.612   1937.32
                 3   1827.800  1418.796        1360.876  1202.004   1936.80
                 5   2082.312  1196.932        1121.884   982.676   1518.40
                 10  2586.232   851.384         681.652   552.988    788.68
                 15  2535.924   694.268         458.224   277.400    379.80
lr5              2   5401.548  5222.544        4775.464  4971.432   5158.32
                 3   5315.208  4752.412        4401.440  5007.588   5853.20
                 5   5243.020  4069.268        3772.132  4801.188   4812.52
                 10  5488.900  2773.692        2561.648  3413.412   3162.64
                 15  6221.272  1854.956        1670.116  2503.704   1936.76
lr5-0.1          2    881.416   485.096         480.432   338.836    526.12
                 3    907.088   338.632         331.076   225.828    264.32
                 5    780.300   158.824         117.388    48.220     44.96
                 10   492.636    36.216          11.104    71.872    142.32
                 15   488.580     6.548           2.016   166.268    170.32
noisy10-0001     2   5855.764  5751.324        5354.672  5564.184   5991.20
                 3   5805.412  5527.984        5167.532  5672.544   6207.48
                 5   5758.980  5202.720        4897.220  5738.064   6078.12
                 10  5596.988  4659.872        4473.416  5537.756   5948.60
                 15  5484.512  4302.984        4172.204  5327.052   5467.52
noisy10-0001-0.1 2   1385.300  1048.852        1036.328   862.316   1320.36
                 3   1496.448   912.300         877.020   736.556   1181.76
                 5   1739.756   693.500         633.368   511.584    723.36
                 10  1843.624   410.768         268.860   154.936    205.32
                 15  1333.096   291.420         135.440   109.836    158.24
noisy10-001      2   5845.100  5738.904        5

In [431]:
grouped_df = synth_l2.groupby(['Dataset', 'k', 'Algorithm']).mean()
grouped_df['Error'] = grouped_df['Error'].apply(lambda x: np.power(x, 1.0 / 2.0))
grouped_df['Error'].unstack()

Algorithm             binmf l2        bmf  bmf+ (int, l2)   mpbmf l2  \
Dataset          k                                                     
full             2   76.852846  75.738973       73.371466  77.894210   
                 3   76.483253  74.270856       72.106144  81.010567   
                 5   76.200709  72.145852       70.458527  83.779568   
                 10  75.161985  68.647331       67.627598  84.340382   
                 15  74.388924  66.381262       65.754209  83.606292   
full0.1          2   43.986180  35.909052       35.014054  35.016396   
                 3   46.548813  35.805363       34.903008  35.081562   
                 5   47.952476  35.571618       34.656428  35.617075   
                 10  42.280019  34.945100       33.926332  37.027233   
                 15  43.312769  34.300496       33.227218  41.726730   
lr10             2   76.671690  75.771947       73.212239  78.168664   
                 3   76.377981  74.282730       71.868574  81.820853   
                 5   76.090762  72.095270       70.040303  84.745855   
                 10  75.060616  68.171431       66.948428  87.534565   
                 15  74.349311  65.537226       64.692040  87.902150   
lr10-0.1         2   37.111400  32.090497       31.679646  29.020820   
                 3   39.394974  29.838968       29.371142  26.990146   
                 5   43.393317  25.844303       25.010398  22.873478   
                 10  43.498506  19.301088       16.173311  14.733771   
                 15  34.987198  16.035336       11.985992  12.409029   
lr15             2   76.674950  75.714886       73.290218  77.602809   
                 3   76.361404  74.263585       72.126167  80.626695   
                 5   76.106688  72.166280       70.407272  83.339234   
                 10  75.133082  68.632674       67.602456  84.259480   
                 15  74.298614  66.345249       65.713286  83.503677   
lr15-0.1         2   41.878157  39.370446       38.912363  36.476568   
                 3   42.878153  37.666909       36.949804  34.932907   
                 5   46.264371  34.596705       33.588689  32.054142   
                 10  53.200827  29.178485       26.877128  26.311442   
                 15  53.342703  26.348966       22.716954  22.159422   
lr5              2   73.539404  72.267171       69.120619  75.248017   
                 3   73.052584  68.937740       66.447272  80.015224   
                 5   72.888380  63.790814       61.832548  85.768456   
                 10  77.634193  52.665852       51.455184  77.885069   
                 15  87.479781  43.069200       42.180375  67.898336   
lr5-0.1          2   29.809797  22.024895       21.710550  18.503513   
                 3   30.447463  18.401956       18.161057  15.408958   
                 5   28.371183  12.602539       11.314769   8.507173   
                 10  22.260189   6.017973        4.045738   8.898090   
                 15  22.119042   2.558906        1.735511  15.148465   
noisy10-0001     2   76.587858  75.837484       73.182840  77.836393   
                 3   76.246364  74.350414       71.872693  80.877933   
                 5   75.937211  72.129883       70.015570  84.188265   
                 10  74.905087  68.263255       66.907877  86.571566   
                 15  74.216332  65.597134       64.711452  87.099920   
noisy10-0001-0.1 2   37.299062  32.385985       32.079090  29.463808   
                 3   39.035676  30.204304       29.837828  27.402263   
                 5   42.677629  26.334388       25.422431  23.504723   
                 10  45.131630  20.267412       17.164498  15.879547   
                 15  37.465824  17.071028       12.992613  13.483768   
noisy10-001      2   76.494366  75.755554       73.103789  77.833720   
                 3   76.217951  74.302813       71.765479  80.917588   
                 5   75.948035  71.998000       69.933740  84.247896   
                 10  74.938028  68.18122

In [430]:
grouped_df = synth_l5.groupby(['Dataset', 'k', 'Algorithm']).mean()
grouped_df['Error'] = grouped_df['Error'].apply(lambda x: np.power(x, 1.0 / 5.0))
grouped_df['Error'].unstack()

Algorithm            binmf l5       bmf  bmf+ (int, l5)  mpbmf l5  nimfa l5
Dataset          k                                                         
full             2   5.709662  5.645830        5.573243  6.623580  5.916772
                 3   5.689753  5.601797        5.536553  7.190142  6.589811
                 5   5.680013  5.537128        5.484059  7.719604  7.237889
                 10  5.661819  5.428120        5.396488  8.192812  7.335222
                 15  5.655592  5.355724        5.336094  8.383578  7.145863
full0.1          2   4.542830  4.188722        4.148468  4.148800  4.316899
                 3   4.822656  4.183880        4.141375  4.152558  4.456946
                 5   5.063126  4.172933        4.129251  4.192450  4.511664
                 10  4.802219  4.143378        4.095769  4.410590  4.346528
                 15  4.960931  4.112635        4.057893  5.122604  4.157290
lr10             2   5.699014  5.646813        5.569297  6.688777  5.732525
                 3   5.686743  5.602155        5.528542  7.344077  6.667270
                 5   5.675929  5.535575        5.471720  7.929253  6.935350
                 10  5.663337  5.413037        5.375428  8.705329  7.327559
                 15  5.669851  5.328380        5.302143  9.059634  7.263842
lr10-0.1         2   4.295613  4.004521        3.986817  3.906272  4.210570
                 3   4.537224  3.889677        3.861228  3.913300  4.334765
                 5   5.017272  3.672366        3.620641  4.009149  4.231611
                 10  5.382734  3.267629        3.060302  4.305219  3.883186
                 15  4.531693  3.034109        2.721945  4.088766  3.373991
lr15             2   5.702608  5.645111        5.572802  6.592169  5.843531
                 3   5.686066  5.601578        5.537248  7.155039  6.715910
                 5   5.676892  5.537755        5.483563  7.685152  7.238682
                 10  5.661165  5.427657        5.395445  8.209804  7.438783
                 15  5.655606  5.354562        5.334341  8.391509  7.267310
lr15-0.1         2   4.455199  4.345784        4.323074  4.283166  4.632281
                 3   4.567936  4.269569        4.234036  4.310061  4.942165
                 5   4.968499  4.126805        4.080082  4.419670  4.930579
                 10  5.841593  3.855006        3.751995  4.913029  4.762278
                 15  6.076676  3.700884        3.518433  5.212544  4.426069
lr5              2   5.598305  5.540850        5.444594  6.872751  5.619429
                 3   5.627383  5.437294        5.360723  7.900090  7.457598
                 5   5.808194  5.271124        5.203323  8.981801  7.186115
                 10  7.176545  4.882156        4.837851  8.909599  6.236612
                 15  8.633325  4.504709        4.472501  8.611140  5.263301
lr5-0.1          2   3.972705  3.444812        3.433576  3.301082  3.526390
                 3   4.133373  3.205868        3.193134  3.323270  3.349374
                 5   4.146156  2.755391        2.652291  3.331675  3.132798
                 10  3.517180  2.050124        1.794100  2.938873  3.704769
                 15  3.463865  1.456202        1.298531  4.812589  3.272854
noisy10-0001     2   5.697702  5.648766        5.568971  6.647344  5.732496
                 3   5.682947  5.604197        5.529245  7.235768  6.675025
                 5   5.672116  5.536638        5.472085  7.870074  7.047361
                 10  5.659962  5.415952        5.374807  8.614149  7.344876
                 15  5.672041  5.330328        5.300685  8.983674  7.179427
noisy10-0001-0.1 2   4.302347  4.019230        4.005295  3.939872  4.295237
                 3   4.529464  3.908657        3.881584  3.943650  4.317222
                 5   4.965911  3.700064        3.652266  4.095703  4.247288
                 10  5.556934  3.332110        3.148063  4.424451  4.017441
                 15  4.808042  3.111028        2.830090  4.217316  3.493876
noisy10-001      2   5.685240  5.646324        5